In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [27]:
from datasets import load_dataset
s = load_dataset('Saptarshi7/RadQA', field='data')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/803 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [33]:
key= 0
with open('../../data/RadQA/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0/train.json', encoding="utf-8") as f:
    radqa = json.load(f)
    for article in radqa["data"]:
        title = article.get("title", "")
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]  # do not strip leading blank spaces GH-2585
            for qa in paragraph["qas"]:
                is_impossible = qa["is_impossible"]

                answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                answers = [answer["text"] for answer in qa["answers"]]
                # Features currently used are "context", "question", and "answers".
                # Others are extracted here for the ease of future expansions.
                print(key, {
                    "title": title,
                    "context": context,
                    "question": qa["question"],
                    "is_impossible": is_impossible,
                    "id": qa["id"],
                    "answers": {
                        "answer_start": answer_starts,
                        "text": answers,
                    },
                })
                break

0 {'title': '796653', 'context': 'IMPRESSION:  Subdural hematomas with blood products of different ages.\n Question vescular abnormality in left suprasellar space.  Findings were\n discussed with Dr. [**Last Name (STitle) 8620**] at 9:25 am on [**2191-8-5**].  An MRI of the brain and MRA\n of the COW is recommended.', 'question': 'Is there any significant change in bleeding?', 'is_impossible': True, 'id': '796653_2_1_I', 'answers': {'answer_start': [], 'text': []}}
0 {'title': '796653', 'context': 'WET READ: MES FRI [**2191-8-5**] 1:40 AM\n  no significant change in hemorrhage\n ______________________________________________________________________________\n                                 FINAL REPORT\n INDICATION: known subarachnoid subdural hemorrhage from outside hospital.\n Evaluate for any change.\n\n TECHNIQUE: Noncontrast head CT.\n\n COMPARISON: (CT done several hours earlier at [**Hospital 539**] Hospital).  At the time\n of attending review, the prior exam is not available f

0 {'title': '1204783', 'context': 'WET READ: JEKh SUN [**2114-9-2**] 9:02 PM\n  no evidence of pneumonia.\n ______________________________________________________________________________\n                                 FINAL REPORT\n CHEST RADIOGRAPH\n\n INDICATION:  spiking fevers, evaluation for pneumonia.\n\n COMPARISON:  [**2114-8-28**].\n\n FINDINGS:  As compared to the previous radiograph, there is no relevant\n change.  Borderline size of the cardiac silhouette without evidence of\n pulmonary edema.  No pleural effusions.  No evidence of pneumonia.', 'question': 'Is there evidence of pneumonia?', 'is_impossible': False, 'id': '1204783_2_1_O', 'answers': {'answer_start': [491], 'text': ['No evidence of pneumonia']}}
0 {'title': '895693', 'context': 'IMPRESSION:  No evidence of deep venous thrombosis in the right lower\n extremity.', 'question': 'Are there any blood clots in the deep veins of right lower limbs?', 'is_impossible': False, 'id': '895693_1_1_I', 'answers': {'answer_